In [8]:
import pandas as pd
import re
import difflib
import os
from difflib import SequenceMatcher
import subprocess
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import statistics as s
from sklearn.utils import shuffle
import nltk
from nltk.tokenize import word_tokenize
import pickle
import numpy as np

In [2]:
df = pd.read_csv('df_centuries.csv')
df['old index'] = df.index.values.tolist()

In [3]:
df = shuffle(df).reset_index(drop=True)
df = df.head(100)
#df = df.sample(n = 100)
#df = df.reset_index(drop=True)
print(df['source'].unique())
print(df['century'].unique())

['Meertens' 'books' 'newspapers' 'anp']
['1600s' '1700s' '1800s' '1900s']


In [4]:
#df['reduced'] = [0]*len(df)
#df['ocr text short'] = ['-']*len(df)

reduced = []

# reduce size of books
def reduce_size(text):
    pages = 5 # pick amount of pages
    characters =  3000 * pages
    #print('test:', text[0:characters])
    if len(text) > characters:
        first_pages = text[0:characters].split(".")
        #print('startthing:', first_pages)
        first_pages = '.'.join(first_pages[:-1])
        #print('first pages reduced:', first_pages)
        #reduced = 1
        reduced.append(1)
    else:
        first_pages = text
        #reduced = 0
        reduced.append(0)
        #print('first pages not reduced:', first_pages)
    #print(type(first_pages))
    #print('end:', first_pages, reduced)
    return first_pages

#df['gt text'], df['reduced'] = df['gt text'].apply(lambda x: reduce_size(x))
#df['ocr text short'], df['reduced'] = df['ocr text'].apply(lambda x: reduce_size(x))
df['gt text org'] = df['gt text']
df['ocr text org'] = df['ocr text']
df['gt text'] = df['gt text'].apply(lambda x: reduce_size(x))
df['ocr text'] = df['ocr text'].apply(lambda x: reduce_size(x))
#df['gt text'], reduced = df['gt text'].apply(lambda x: reduce_size(x))
#df['ocr text'], reduced = df['ocr text'].apply(lambda x: reduce_size(x))
    

In [5]:
print(reduced)
#df.head()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
df_17th = df[df['century'] == '1600s']
df_18th = df[df['century'] == '1700s']
df_19th = df[df['century'] == '1800s']
df_20th = df[df['century'] == '1900s']


In [7]:
#datasets = [df_17th, df_18th, df_19th, df_20th] #datasets[0] == df_17th etc.
#datasets = [df_17th.head(), df_18th.head(), df_19th.head(), df_20th.head()]
#print(len(df_17th), len(df_18th),len(df_19th),len(df_20th))

In [8]:
# column selection
print(df.columns)
#df = df[['identifier', 'gt text', 'ocr text', 'CER', 'WER', 'source', 'word count gt',
#       'word count ocr', 'century']]

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org'],
      dtype='object')


In [9]:
print(df['source'].unique())
print(df['century'].unique())

['Meertens' 'books' 'newspapers' 'anp']
['1600s' '1700s' '1800s' '1900s']


In [9]:
def find_names(text, names):
    NNP_tags = "ik je jij jou u hij zij ze het wij we jullie mij me hem haar ons hen hun".split(' ')
    text = word_tokenize(text)
    tags = nltk.pos_tag(text)
    
    for tag in tags:
        if (tag[1] == 'NNP'):
            # we want words that are NN(P)s, but are not in NNP_tags
            NN = tag[0]
            NN = NN.lower()
            if NN not in NNP_tags:
                names.append(NN)
    return names

def preprocessing(text):
    text = re.sub("[^a-zA-Z0-9-. ]+", '', text) # remove punctuation except for hyphens and dots
    text = text.lower() # lowercase
    return text

#for i in range(len(datasets)):
#    # preprocess identifier
#    datasets[i]['identifier'] = datasets[i]['identifier'].apply(lambda x: str(x))
#    datasets[i]['identifier'] = datasets[i]['identifier'].apply(lambda x: x.replace('_', ':'))
#    datasets[i]['identifier'] = datasets[i]['identifier'].apply(lambda x: x + '_None')
    # preprocess OCR and GT
#    datasets[i]['gt text'] = datasets[i]['gt text'].apply(lambda x: preprocessing(x))
#    datasets[i]['ocr text'] = datasets[i]['gt text'].apply(lambda x: preprocessing(x))

#df['gt text org'] = df['gt text']
#df['ocr text org'] = df['ocr text']

# preprocess identifier
df['identifier'] = df['identifier'].apply(lambda x: str(x))
df['identifier'] = df['identifier'].apply(lambda x: x.replace('_', '.'))
df['identifier'] = df['identifier'].apply(lambda x: x.replace(':', '.'))
#df['identifier'] = df['identifier'].apply(lambda x: x + '_None')
# preprocess OCR and GT
df['gt text'] = df['gt text'].apply(lambda x: preprocessing(x))
df['ocr text'] = df['gt text'].apply(lambda x: preprocessing(x))



In [10]:
# create parts for cross-validation, train, validation oand test set
df_17th = df[df['century'] == '1600s']
df_18th = df[df['century'] == '1700s']
df_19th = df[df['century'] == '1800s']
df_20th = df[df['century'] == '1900s']

datasets = [df_17th, df_18th, df_19th, df_20th]
# create a list of train, validation and test set

train = pd.DataFrame()
val = pd.DataFrame()
test = pd.DataFrame()
for dataset in datasets:
    splits = np.array_split(dataset, 5)
    train_sub = splits[:3]
    train_sub = pd.concat(train_sub)
    val_sub = splits[3]
    test_sub = splits[4]
    train = pd.concat([train, train_sub])
    val = pd.concat([val, val_sub])
    test = pd.concat([test, test_sub])

train_indices = train.index.values.tolist() 
val_indices = val.index.values.tolist()  
test_indices = test.index.values.tolist()  

for index, row in df.iterrows():
    if index in train_indices:
        df.at[index, 'set'] = 'train'
    elif index in val_indices:
        df.at[index, 'set'] = 'val'
    elif index in test_indices:
        df.at[index, 'set'] = 'test'


In [11]:
def fuzzy_matching(row): # create files with matched sentences
    #save_path = r"C:\Users\Gebruiker\Desktop\Thesis\Documenten\Documenten"
    
    ID = row['identifier']
    page = 'None'
    
    # split GT and OCR-output of each row into sentences (based on full stops).
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")

    match_list = []

    for sentence in gt_sentences:
        for match in ocr_sentences:
            if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                match_list.extend([[sentence, match]])
                
    matched_gt_sentences = []
    matched_ocr_sentences = []

    for sentence in match_list:
        if len(sentence[0]) or len(sentence[1]) > 0:
            matched_gt_sentences.append(sentence[0])
            matched_ocr_sentences.append(sentence[1])
    
    # .join() with lists
    separator = '.'
    doc_GT = separator.join(matched_gt_sentences)
    doc_OCR = separator.join(matched_ocr_sentences)
    
    #return matched_gt_sentences, matched_ocr_sentences
    return doc_GT, doc_OCR

In [12]:
jar_file = "ocrevalUAtion-1.3.4-jar-with-dependencies.jar"

def evaluation(index, row, doc_GT, doc_OCR):
    ID = row['identifier']
    page = 'None'
    
    doc_OCR = doc_OCR.replace('.', '')
    doc_OCR = re.sub(' +', ' ', doc_OCR)
    filename_ocr = f"{ID}_{page}_OCR.txt"
    #file_ocr = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_ocr = open(filename_ocr,"w+", encoding="utf-8")
    file_ocr.write(doc_OCR)
    file_ocr.close()
    
    doc_GT = doc_GT.replace('.', '')
    doc_GT = re.sub(' +', ' ', doc_GT)
    filename_gt = f"{ID}_{page}_GT.txt"
    #file_gt = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_gt = open(filename_gt,"w+", encoding="utf-8")
    file_gt.write(doc_GT)
    file_gt.close()
    
    #output = ID + '_' + page + ".html"
    output = f"{ID}_{page}.html"
    
    #process = subprocess.call("/home/nvanthof/jdk-16.0.1/bin/java -cp " + jar_file  + " eu.digitisation.Main -gt " + filename_gt + " -ocr "+ filename_ocr +" -o " + output + "")
    #os.system("/home/nvanthof/jdk-16.0.1/bin/java -cp /home/nvanthof/ocrevalUAtion-1.3.4-jar-with-dependencies.jar eu.digitisation.Main -gt /home/nvanthof/ddd.010728187.mpeg21.a0005_None_GT.txt -ocr /home/nvanthof/ddd.010728187.mpeg21.a0005_None_OCR.txt  -o /home/nvanthof/OUTPUT2.html")
    command = f"/home/nvanthof/jdk-16.0.1/bin/java -cp /home/nvanthof/ocrevalUAtion-1.3.4-jar-with-dependencies.jar eu.digitisation.Main -gt /home/nvanthof/{filename_gt} -ocr /home/nvanthof/{filename_ocr}  -o /home/nvanthof/{output}"
    os.system(command)
    sleep(5)
    
    soup = BeautifulSoup(open(output, encoding='utf-8'))
    table = soup.find("table", attrs={'border': '1'})
    # Split the filename, and extract the identifier and pagenr together as identifier 
    # Find the first table (this is the table in which the scores are stored)
    # Find the tags in which 'CER', 'WER', and 'WER (order independent)' are stored and take the next tag to get the score 
    cer = table.find('td', text='CER')
    cerScore = cer.findNext('td')
    wer = table.find('td', text='WER')
    werScore = wer.findNext('td')
    werOI = table.find('td', text='WER (order independent)')
    werOIScore = werOI.findNext('td')
    
    os.remove(filename_gt)
    os.remove(filename_ocr)
    os.remove(output)
    return float(cerScore.text), float(werScore.text)   
    
    return cerScore.text, werScore.text

In [13]:
for index, row in df.loc[df['set'].isin(['train', 'val'])].iterrows():
    print(index)
    if index%1000 == 0:
        print(index)
    doc_GT, doc_OCR = fuzzy_matching(row)
    #GT_matched_docs_list.append(matched_gt_sentences)
    #OCR_matched_docs_list.append(matched_ocr_sentences)
    df.at[index, 'gt sentences matched'], df.at[index, 'ocr sentences matched'] = doc_GT, doc_OCR

for index, row in df.loc[df['set'].isin(['test'])].iterrows():
    print(index)
    if index%1000 == 0:
        print(index)
    doc_GT, doc_OCR = fuzzy_matching(row)
    #GT_matched_docs_list.append(matched_gt_sentences)
    #OCR_matched_docs_list.append(matched_ocr_sentences)
    df.at[index, 'gt sentences matchated'], df.at[index, 'ocr sentences matched'] = doc_GT, doc_OCR
    df.at[index, 'CER matched sentences'], df.at[index, 'WER matched sentences'] = evaluation(index, row, doc_GT, doc_OCR)
    

        


0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
51
52
53
54
55
56
57
58
59
60
62
63
95
50
61
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
96
97
98
99


In [14]:
# count sentences (longer than 4 words) in GT and OCR
# count words in sentences longer than 4 words

#meta_df = pd.DataFrame(columns=['identifier', 'sentences gt', 'sentences ocr', 'avg sentence length gt', ' avg sentence length ocr'])
meta_df = pd.DataFrame()

def sentence_count(row):
# count of sentences in GT >= 4 words
    sentences_gt = 0
    words_GT_sentences = []
    #characters_GT_sentences = []
    for k in row['gt text'].split('.'):
        k = [word for word in k.split(' ') if len(word) > 1]
        if len(k) >= 4:
            sentences_gt += 1
            words_GT_sentences.append(len(k))
            #characters_GT_sentences.append(len(' '.join(k)))
    # count of sentences in OCR >= 4 words
    sentences_ocr = 0
    words_OCR_sentences = []
    #characters_OCR_sentences = []
    for k in row['ocr text'].split('.'):
        k = [word for word in k.split(' ') if len(word) > 1]
        if len(k) >= 4:
            sentences_ocr += 1
            words_OCR_sentences.append(len(k))
            #characters_OCR_sentences.append(len(' '.join(k)))

    return sentences_gt, sentences_ocr, words_GT_sentences, words_OCR_sentences


#df['gt text'] = df['gt text'].apply(lambda x: preprocessing(x))

for index, row in df.iterrows(): 
    sentences_gt, sentences_ocr, words_GT_sentences, words_OCR_sentences = sentence_count(row)
    meta_df = meta_df.append({'identifier': row['identifier'], \
                              'sentences gt (fuzzy matched)': sentences_gt, 'sentences ocr (fuzzy matched)': sentences_ocr, \
                              'avg sentence length gt (fuzzy matched)': s.mean(words_GT_sentences), 'avg sentence length ocr (fuzzy matched)': s.mean(words_OCR_sentences), \
                              'max sentence length gt (fuzzy matched)': max(words_GT_sentences), 'max sentence length ocr (fuzzy matched)': max(words_OCR_sentences), \
                              'word count gt (fuzzy matched)': sum(words_GT_sentences), 'word count ocr (fuzzy matched)': sum(words_OCR_sentences)}, ignore_index = True)
    
df = pd.merge(left=df, right=meta_df, on='identifier')   

In [17]:
pd.set_option('display.max_columns', None)
df

,Unnamed: 0,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year,century,old index,gt text org,ocr text org,set,gt sentences matched,ocr sentences matched,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
0,5917,ddd.010680629.mpeg21.a0005,wt augs-burgh den 21 dito. dat den generael co...,wt augs-burgh den 21 dito. dat den generael co...,52.71,91.59,89.72,91.460000,40.74,0.062893,0.52,Meertens,106,89,16--,1600s,5917,Wt Augs-burgh den 21 dito. Dat den Generael Co...,Wt Augf-burgh den 21 dito. <©ot ben «öèneroef ...,train,NaN,NaN,NaN,NaN,17.500000,17.500000,34.0,34.0,6.0,6.0,105.0,105.0
1,29781,ddd.011225496.mpeg21.a0001,italien. turin den 10 agusti. de verkochte gec...,italien. turin den 10 agusti. de verkochte gec...,15.36,44.61,37.84,71.230000,60.91,0.310000,0.14,Meertens,394,399,16--,1600s,29781,ITALIEN. TUrin den 10 Agusti. De verkochte gec...,ITALIEN. TUrin den 10 Augufti. De verkochte ge...,train,NaN,NaN,NaN,NaN,17.761905,17.761905,78.0,78.0,21.0,21.0,373.0,373.0
2,32893,ddd.011226576.mpeg21.a0002,switserlant. basel den 17 july. zurich bern en...,switserlant. basel den 17 july. zurich bern en...,55.54,97.93,92.23,80.330000,32.85,0.070000,0.54,Meertens,190,147,16--,1600s,32893,"SWITSERLANT. Basel den 17 July. Zurich, Bern e...",SWITSERLANT. rÃ¯ SfiCTjCOteaodcje pan? ton| fe...,train,NaN,NaN,NaN,NaN,37.800000,37.800000,87.0,87.0,5.0,5.0,189.0,189.0
3,34154,ddd.011226878.mpeg21.a0006,nederlanden. s gravenhage den 8 maert. den env...,nederlanden. s gravenhage den 8 maert. den env...,19.65,54.26,47.54,71.270000,51.47,0.220000,0.18,Meertens,1046,1128,16--,1600s,34154,NEDERLANDEN. \'s Gravenhage den 8 Maert. Den E...,NEDERLANDEN. 's Gravcnhage den 8 Macrt. â– De...,train,NaN,NaN,NaN,NaN,25.871795,25.871795,116.0,116.0,39.0,39.0,1009.0,1009.0
4,12000,ddd.010759113.mpeg21.a0003,poolen pruysen ampc. warschauw den 18 february...,poolen pruysen ampc. warschauw den 18 february...,14.12,44.74,41.44,79.870000,56.50,0.260000,0.13,Meertens,634,621,16--,1600s,12000,"POOLEN PRUYSEN, &amp;c. Warschauw den 18 Febru...","POOLEN PRUYSEN, &c. WarfchaÃ¼,v den 18 Febiuai...",train,NaN,NaN,NaN,NaN,38.500000,38.500000,126.0,126.0,16.0,16.0,616.0,616.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40763,537354,rado i. eerste uitzending van 27 september i93...,rado i. eerste uitzending van 27 september i93...,10.90,27.04,22.88,75.477707,82.73,0.440000,0.12,anp,527,571,1937,1900s,38763,RAD�O I. Eerste uitzending van 27 September I9...,"b' 0 I. lorste uitzen"" 27 Sc. Copie Bi). Door ...",train,NaN,NaN,NaN,NaN,20.960000,20.960000,48.0,48.0,25.0,25.0,524.0,524.0
96,16523,ddd.010926739.mpeg21.a0002,vranckryck. parys den 4 february. den paus hee...,vranckryck. parys den 4 february. den paus hee...,75.99,110.86,104.47,79.190000,36.59,0.070000,0.68,Meertens,308,270,16--,1600s,16523,VRANCKRYCK. Parys den 4 February. Den Paus hee...,VRANCKRYCK. I 7: ÃŠsrys dra <t February.7* Ben...,test,vranckryck. parys den 4 february. parys den 4 ...,vranckryck. parys den 4 february. parys den 4 ...,0.39,1.2,36.500000,36.500000,82.0,82.0,8.0,8.0,292.0,292.0
97,29516,ddd.011225452.mpeg21.a0004,engelant ampc. dublin den 13 maert. de stad li...,engelant ampc. dublin den 13 maert. de stad li...,13.28,40.06,36.55,79.290000,67.87,0.340000,0.12,Meertens,335,353,16--,1600s,29516,"ENGELANT, &amp;c. Dublin den 13 Maert. De Stad...","ENGELANT, &c. Dublin den 13 Maert. De Stad Lim...",test,engelant ampc. dublin den 13 maert. de stad li...,engelant ampc. dublin den 13 maert. de stad li...,0.00,0.0,1

In [18]:
# sum of values
df.groupby('century').sum()

,Unnamed: 0,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,word count gt,word count ocr,old index,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
century,,,,,,,,,,,,,,,,,,,,,
1600s,1882289,3184.38,6591.53,6022.59,7299.290000,4721.67,17.234787,29.740000,47228,42929,1882289,5.59,12.27,2644.617180,2644.617180,8932.0,8932.0,1585.0,1585.0,44653.0,44653.0
1700s,2735,100.40,151.31,139.17,350.650000,342.72,2.360000,0.970000,1589,987,141967,0.00,0.00,240.310606,240.310606,617.0,617.0,29.0,29.0,1518.0,1518.0
1800s,1533,11.94,24.34,19.45,88.430000,77.67,0.542473,0.119155,2502,2496,37908,0.00,0.00,11.569536,11.569536,61.0,61.0,151.0,151.0,1747.0,1747.0
1900s,83137,28.11,78.70,68.74,241.714003,246.04,1.715849,0.300009,4598,4678,115512,0.00,0.00,57.207788,57.207788,184.0,184.0,150.0,150.0,3004.0,3004.0


In [19]:
# mean of values
df.groupby('century').mean()

,Unnamed: 0,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,word count gt,word count ocr,old index,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched)
century,,,,,,,,,,,,,,,,,,,,,
1600s,20459.663043,34.612826,71.647065,65.462935,79.340109,51.322500,0.187335,0.323261,513.347826,466.619565,20459.663043,0.155278,0.340833,28.745839,28.745839,97.086957,97.086957,17.228261,17.228261,485.358696,485.358696
1700s,683.750000,25.100000,37.827500,34.792500,87.662500,85.680000,0.590000,0.242500,397.250000,246.750000,35491.750000,0.000000,0.000000,60.077652,60.077652,154.250000,154.250000,7.250000,7.250000,379.500000,379.500000
1800s,1533.000000,11.940000,24.340000,19.450000,88.430000,77.670000,0.542473,0.119155,2502.000000,2496.000000,37908.000000,NaN,NaN,11.569536,11.569536,61.000000,61.000000,151.000000,151.000000,1747.000000,1747.000000
1900s,27712.333333,9.370000,26.233333,22.913333,80.571334,82.013333,0.571950,0.100003,1532.666667,1559.333333,38504.000000,NaN,NaN,19.069263,19.069263,61.333333,61.333333,50.000000,50.000000,1001.333333,1001.333333


In [15]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == '' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,10):
                        if i-j >= 0:
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i-j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i+j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            try:
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                    pass
            except:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before forwards match GT:', GT)
                #print('before forwards bad match OCR:', OCR)
                if i-1 >= 0:
                    GT.insert(i, '')
                else:
                    GT.insert(0, '')
                if i+1 <= len(GT):
                    OCR.insert(i+1, '')
                else:
                    GT.append('')
                #print('after forwards match GT:', GT)
                #print('after forwards bad match OCR:', OCR)
                
                continue
    
   #print('after forwards GT:', GT)
    #print('after forwards OCR:', OCR)
    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')
    
    if True:
        
        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
            #print('i:', i)
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.65:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,len(GT)):
                            if j >= 10:
                                extra_ratio = 0.1
                            else:
                                extra_ratio = 0
                            if i-j >= -len(OCR):
                                if j >= 5:
                                    if (len(GT[i]) >= 5) or (len(OCR[i-j]) >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i-j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    if i+k < 0:
                                        GT.insert(i+1, '')
                                    else: 
                                        GT.append('')
                                break
                            if i+j <= -1:
                                if j >= 5:
                                    if (GT[i] >= 5) or (OCR[i+j] >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i+j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.30:
                pass
            else:
                #print('bad match', GT[i], OCR[i])
                #print('before backwards bad match GT:',  GT)
                #print('before backwards bad match OCR:',  OCR)
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    GT.insert(i, '')
                    OCR.insert(i+1, '')
                #print('after backwards bad match GT:',  GT)
                #print('after backwards bad match OCR:',  OCR)
                
                
    #print('after backwards GT:',  GT)
    #print('after backwards OCR:',  OCR)
                    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')


    for i in range(len(GT)-1):
        try:
            #print(i)
            #print(GT[i], OCR[i])
            if (GT[i] == '') and (OCR[i] == ''):
                del GT[i]
                del OCR[i]
        except:
            break
            
    try:
        if SequenceMatcher(None, GT[-1], OCR[-2]).ratio() >= 0.65 and ((GT[-1] != '') or ((OCR[-2] != ''))):
            #print('FOUND:', GT[-1], OCR[-2])
            OCR.insert(-2, '')
            GT.append('')
    except:
        pass
    try:
        if SequenceMatcher(None, GT[-2], OCR[-1]).ratio() >= 0.65 and ((GT[-2] != '') or ((OCR[-1] != ''))):
            #print('FOUND:', OCR[-1], GT[-2])
            GT.insert(-2, '')
            OCR.append('')
    except:
        pass
    
    for i in range(len(OCR)):
        if GT[i] == '' and OCR[i] == '':
            GT[i] = 'REMOVE'
            OCR[i] = 'REMOVE'

    GT = list(filter(lambda a: a != 'REMOVE', GT))
    OCR = list(filter(lambda a: a != 'REMOVE', OCR))
    
    
    #print('end GT:',  GT)
    #print('end OCR:',  OCR)
    return GT, OCR

In [16]:
# verify word alignment quality
def verify_alignment(GT, OCR):
    
    if len(GT) != len(OCR):
        alignment_validation = False
        return alignment_validation
    
    min_word_length = 8 # the higher this number, the more likely a alignment will be seen as valid, thus higher chance at false positive, lower chance at false negative. 
    match_ratio = 0.70
    anchors_list = []
    for i in range(len(GT)):
        if len(GT[i]) >= min_word_length:  
            anchors_list.append(GT[i])
        if len(OCR[i]) >= min_word_length:
            anchors_list.append(OCR[i])
    #print(anchors_list)


    alignment_validation = True

    for anchor in anchors_list:
        #print('ANCHOR:', anchor)
        matches_GT = set()
        matches_OCR = set()
        match = False
        for i in range(len(GT)):
            if (SequenceMatcher(None, GT[i], anchor).ratio() >= 0.65) and (len(GT[i]) >= min_word_length or len(GT[i]) == 0):
                matches_GT.add(i)
                #print('added as GT match:', GT[i])
            if SequenceMatcher(None, OCR[i], anchor).ratio() >= 0.65 and (len(OCR[i]) >= min_word_length or len(OCR[i]) == 0):
                matches_OCR.add(i)
                #print('added as OCR match:', OCR[i])
        #print('anchor:', anchor)
        #print(matches_GT)
        #print(matches_OCR)
        if matches_GT == matches_OCR:
            match = True
        if match == False:
            match = True
            points_set1 = set() # counts when the matched opposite word is '' for OCR
            points_set2 = set() # counts when the matched opposite word is '' for GT
            for index in matches_GT.difference(matches_OCR): # elements in matches_GT that are not in matches_OCR
                if OCR[index] == '':
                    points_set1.add(index)
                if OCR[index] != '' and len(matches_OCR.difference(matches_GT)) != 0:
                    match = False
            for index in matches_OCR.difference(matches_GT): # elements in matches_OCR that are not in matches_GT
                if GT[index] == '':
                    points_set2.add(index)
                if GT[index] != '' and len(matches_GT.difference(matches_OCR)) != 0:
                    match = False
            if len(points_set1.intersection(points_set2)) != 0: # if an anchor words was match in both OCR and GT with an '', it's a bad match.
                match = False
        if match == False:
            for index in (matches_GT.difference(matches_OCR)):
                index += 1
                if index in matches_OCR.difference(matches_GT):
                    if (GT[index-1] == '' or OCR[index-1] == '') and (SequenceMatcher(None, GT[index], OCR[index]).ratio() >= match_ratio):
                        match = True
            for index in (matches_OCR.difference(matches_GT)):
                index += 1
                if index in matches_GT.difference(matches_OCR):
                    if (GT[index] == '' or OCR[index] == '') and (SequenceMatcher(None, GT[index-1], OCR[index-1]).ratio() >= match_ratio):
                        match = True

        if match == False:
            alignment_validation = False # bad alignment

    return alignment_validation

In [18]:
list_aligned_GT_docs = []
list_aligned_OCR_docs = []
good_alignments_doc = []
bad_alignments_doc = []

for index, row in df.loc[df['set'].isin(['val','test'])].iterrows():
#for index, row in df.iterrows():
    GT_sentences = row['gt sentences matched'].split('.')
    OCR_sentences = row['ocr sentences matched'].split('.')
    good_alignments = 0
    bad_alignments = 0
    aligned_GT = []
    aligned_OCR = []
    for i in range(len(GT_sentences)):
        GT = GT_sentences[i].split(' ')
        OCR = OCR_sentences[i].split(' ')
        GT, OCR = word_alignment(GT, OCR)
        verification = verify_alignment(GT, OCR)
        if verification == False: # bad alignment
            GT = ['REMOVED'] * len(GT)
            bad_alignments += 1
        else:
            if len(OCR) >= 4:
                good_alignments += 1
        aligned_GT.append(GT)
        aligned_OCR.append(OCR)
    list_aligned_GT_docs.append(aligned_GT)
    list_aligned_OCR_docs.append(aligned_OCR)
    good_alignments_doc.append(good_alignments)
    bad_alignments_doc.append(bad_alignments)

# this list should only apply to rows in the val and test set
df['aligned_GT_sentences'] = [np.nan] * len(df)
df['aligned_OCR_sentences'] = [np.nan] * len(df)
df['good_alignments'] = [np.nan] * len(df)
df['bad_alignments'] = [np.nan] * len(df)
df['aligned_GT_sentences'][val_indices + test_indices] = list_aligned_GT_docs
df['aligned_OCR_sentences'][val_indices + test_indices] = list_aligned_OCR_docs
df['good_alignments'][val_indices + test_indices] = good_alignments_doc
df['bad_alignments'][val_indices + test_indices] = bad_alignments_doc


    
    

/home/nvanthof/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nvanthof/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/nvanthof/.local/lib/python3.6/site-packages/pandas/core/internals/blocks.py:832: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you mus

In [20]:
df.columns

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org', 'set', 'gt sentences matched', 'ocr sentences matched',
       'CER matched sentences', 'WER matched sentences',
       'avg sentence length gt (fuzzy matched)_x',
       'avg sentence length ocr (fuzzy matched)_x',
       'max sentence length gt (fuzzy matched)_x',
       'max sentence length ocr (fuzzy matched)_x',
       'sentences gt (fuzzy matched)_x', 'sentences ocr (fuzzy matched)_x',
       'word count gt (fuzzy matched)_x', 'word count ocr (fuzzy matched)_x',
       'longest_streak', 'avg_longest_streaks', 'avg_total_missing_words',
       'avg_perc_missing_words ', 'gt sentences matchated',
       'avg sentence length gt (fuzzy matched)_

In [34]:
def missing_words_count(row):
    ocr_aligned_sentences = row['aligned_OCR_sentences']
    # analysing the word alignment
    # word alignment for each sentence
    longest_streaks = []
    total_missing_words = []
    perc_missing_words = []
    for sentence in ocr_aligned_sentences: # for each sentence
        streaks = []
        streak = 0
        for word in sentence: # for each word
            if word == '':
                streak = streak + 1
            else:
                streaks.append(streak)
                streak = 0
        try:
            longest_streaks.append(max(streaks))# longest streak of missing word in sentence
            total_missing_words.append(sum(streaks)) # total of missing words in a sentence
            perc_missing_words.append(sum(streaks) / len(ocr_aligned_sentences))
        except ValueError:
            longest_streaks.append(0)# longest streak of missing word in sentence
            total_missing_words.append(0) # total of missing words in a sentence
            perc_missing_words.append(0)
    # statistics word alignment doc level:
    # find longest streak of all longest streaks:
    import statistics as s
    longest_streak = max(longest_streaks)
    avg_longest_streaks = s.mean(longest_streaks)
    avg_total_missing_words = s.mean(total_missing_words)
    avg_perc_missing_words = s.mean(perc_missing_words)
    return longest_streak, avg_longest_streaks, avg_total_missing_words, avg_perc_missing_words 
    
for index, row in df.loc[df['set'].isin(['val','test'])].iterrows():
    df.at[index, 'longest_streak'], df.at[index, 'avg_longest_streaks'], df.at[index, 'avg_total_missing_words'], df.at[index, 'avg_perc_missing_words '] = missing_words_count(row)
          

0.0
0.0
0.0
0.006666666666666666
0.0
0.0
0.0
0.0
0.0017301038062283738
0.0
0.0013850415512465374
0.0
0.0
3.790346128793154e-05
0.0
0.0
0.0
0.0
0.0
0.0
0.009695290858725761
0.007889546351084813
0.0
0.0
0.006802721088435374
0.001953125
0.0004624812117007747
0.0
0.0
0.0
0.0
0.004382761139517896
0.0
0.0
0.0
0.0
0.0


In [32]:
pd.set_option('display.max_rows', None)
df


,Unnamed: 0,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year,century,old index,gt text org,ocr text org,set,gt sentences matched,ocr sentences matched,CER matched sentences,WER matched sentences,avg sentence length gt (fuzzy matched),avg sentence length ocr (fuzzy matched),max sentence length gt (fuzzy matched),max sentence length ocr (fuzzy matched),sentences gt (fuzzy matched),sentences ocr (fuzzy matched),word count gt (fuzzy matched),word count ocr (fuzzy matched),longest_streak,avg_longest_streaks,avg_total_missing_words,avg_perc_missing_words
0,5917,ddd.010680629.mpeg21.a0005,wt augs-burgh den 21 dito. dat den generael co...,wt augs-burgh den 21 dito. dat den generael co...,52.71,91.59,89.72,91.460000,40.74,0.062893,0.520000,Meertens,106,89,16--,1600s,5917,Wt Augs-burgh den 21 dito. Dat den Generael Co...,Wt Augf-burgh den 21 dito. <©ot ben «öèneroef ...,train,NaN,NaN,NaN,NaN,17.500000,17.500000,34.0,34.0,6.0,6.0,105.0,105.0,NaN,NaN,NaN,NaN
1,29781,ddd.011225496.mpeg21.a0001,italien. turin den 10 agusti. de verkochte gec...,italien. turin den 10 agusti. de verkochte gec...,15.36,44.61,37.84,71.230000,60.91,0.310000,0.140000,Meertens,394,399,16--,1600s,29781,ITALIEN. TUrin den 10 Agusti. De verkochte gec...,ITALIEN. TUrin den 10 Augufti. De verkochte ge...,train,NaN,NaN,NaN,NaN,17.761905,17.761905,78.0,78.0,21.0,21.0,373.0,373.0,NaN,NaN,NaN,NaN
2,32893,ddd.011226576.mpeg21.a0002,switserlant. basel den 17 july. zurich bern en...,switserlant. basel den 17 july. zurich bern en...,55.54,97.93,92.23,80.330000,32.85,0.070000,0.540000,Meertens,190,147,16--,1600s,32893,"SWITSERLANT. Basel den 17 July. Zurich, Bern e...",SWITSERLANT. rÃ¯ SfiCTjCOteaodcje pan? ton| fe...,train,NaN,NaN,NaN,NaN,37.800000,37.800000,87.0,87.0,5.0,5.0,189.0,189.0,NaN,NaN,NaN,NaN
3,34154,ddd.011226878.mpeg21.a0006,nederlanden. s gravenhage den 8 maert. den env...,nederlanden. s gravenhage den 8 maert. den env...,19.65,54.26,47.54,71.270000,51.47,0.220000,0.180000,Meertens,1046,1128,16--,1600s,34154,NEDERLANDEN. \'s Gravenhage den 8 Maert. Den E...,NEDERLANDEN. 's Gravcnhage den 8 Macrt. â– De...,train,NaN,NaN,NaN,NaN,25.871795,25.871795,116.0,116.0,39.0,39.0,1009.0,1009.0,NaN,NaN,NaN,NaN
4,12000,ddd.010759113.mpeg21.a0003,poolen pruysen ampc. warschauw den 18 february...,poolen pruysen ampc. warschauw den 18 february...,14.12,44.74,41.44,79.870000,56.50,0.260000,0.130000,Meertens,634,621,16--,1600s,12000,"POOLEN PRUYSEN, &amp;c. Warschauw den 18 Febru...","POOLEN PRUYSEN, &c. WarfchaÃ¼,v den 18 Febiuai...",train,NaN,NaN,NaN,NaN,38.500000,38.500000,126.0,126.0,16.0,16.0,616.0,616.0,NaN,NaN,NaN,NaN
5,34352,ddd.011226926.mpeg21.a0005,polen pruyssen ampc. lemburg den 14 mey. in de...,polen pruyssen ampc. lemburg den 14 mey. in de...,16.57,37.14,37.14,73.080000,75.86,0.460000,0.150000,Meertens,33,35,16--,1600s,34352,"POLEN, PRUYSSEN, &amp;c. Lemburg den 14 Mey. I...","POLEN, PRUYSSEN, &c. ' Lemburg den 14 Mey. In ...",train,NaN,NaN,NaN,NaN,15.000000,15.000000,26.0,26.0,2.0,2.0,30.0,30.0,NaN,NaN,NaN,NaN
6,947,ddd.010634148.mpeg21.a0007,vande mayn-stroom den 1 february. den cheur-vo...,vande mayn-stroom den 1 february. den cheur-vo...,39.23,82.26,80.65,90.570000,48.00,0.127660,0.380000,Meertens,60,51,16--,1600s,947,Vande Mayn-stroom den 1 February. Den Cheur-Vo...,Van de Vlayn-stroom den 1 February. Dm Mftojft...,train,NaN,NaN,NaN,NaN,14.750000,14.750000,45.0,45.0,4.0,4.0,59.0,59.0,NaN,NaN,NaN,NaN
7,10087,ddd.010633885.mpeg21.a0010,brieven van antwerpen melden vanden 23 iulij d...,brieven van antwerpen melden vanden 23 iulij d...,43.46,94.20,87.80,67.660000,35.21,0.040000,0.410000,Meertens,666,621,16--,1600s,10087,"Brieven van Antwerpen melden vanden 23 Iulij, ...",B?feöen ban ainttoerpen meiben banben 23 gulp ...,train,NaN,NaN,NaN,NaN,40.812500,40.812500,122.0,122.0,16.0,16.0,653.0,653.0,NaN,NaN,NaN,NaN
8,27035,ddd.0112

In [ ]:
# find proper pronouns
gt_names = []
ocr_names = []
# these both can look into train, val, and test set
for text in list(df['gt sentences matched']):
    gt_names = find_names(text, gt_names)
for text in list(df['ocr sentences matched']):
    ocr_names = find_names(text, ocr_names)

gt_names = []
for name in gt_names_org:
    if len(name) >= 5:
        gt_names.append(name)
ocr_names = []
for name in ocr_names_org:
    if len(name) >= 5:
        ocr_names.append(name)
        
with open('gt_names.txt', 'wb') as f:
    pickle.dump(gt_names, f)

with open('ocr_names.txt', 'wb') as f:
    pickle.dump(ocr_names, f)
        



[2, 4, 6, 7, 2, 4, 6, 7]


In [35]:
# save dataframe
df.to_csv('preprocessed_df100.csv', index=False)

In [35]:
# to find out longest sentence
print(max(list(df['max sentence length gt (fuzzy matched)'])))

731.0


In [36]:
# save lists:
with open("gt_names.txt", "wb") as fp:   #Pickling
    pickle.dump(gt_names, fp)
with open("ocr_names.txt", "wb") as fp:   #Pickling
    pickle.dump(ocr_names, fp)
#with open("test.txt", "rb") as fp:   # Unpickling
#    b = pickle.load(fp)